# Top End Assembly Vibration Study

As described in [OBS-467], we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). 
The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday). 

Accordingly to [SITCOM-1285], on March 15th (Friday), we ran soak tests using the Rotator at different configurations. 
We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.

In addition to both cases, on Marth 21th (Thursday), we executed [BLOCK-197] a couple of times. 
[OBS-498] reports multiple faults on the hexapods associated with the compensation mode. 
It also indicates that we might have vibrations during its execution, we need to review it. 

Finally, on March 25th, we executed [BLOCK-197] twice. 
First with the rotator in ENABLED. 
Second with the rotator in STANDBY and with its cabinet turned off. 
We want to compare both cases. 

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197
[OBS-467]: https://rubinobs.atlassian.net/browse/OBS-467
[OBS-498]: https://rubinobs.atlassian.net/browse/OBS-498
[SITCOM-1285]: https://rubinobs.atlassian.net/browse/SITCOM-1285

## Notebook Preparation

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from astropy.time import Time, TimeDelta
from lsst_efd_client.efd_helper import EfdClient, merge_packed_time_series

# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)